In [1]:
import torch
from langchain.document_loaders import UnstructuredMarkdownLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loader = UnstructuredMarkdownLoader('data/document.md')
documents = loader.load()
print(documents[0])

page_content='什么是 Top-K 和 Top-P 采样？Temperature 如何影响生成结果？

在之前的文章中我们探讨了 Beam Search 和 Greedy Search。

现在来聊聊 model.generate() 中常见的三个参数: top-k, top-p 和 temperature。

代码文件下载

目录

采样方法概述

Top-K 采样详解

工作原理

数学表述

代码示例

Top-P 采样详解

工作原理

数学表述

代码示例

Temperature 的作用

工作原理

代码示例

在大模型中的应用

Top-K 和 Top-P 采样是否可以一起使用？

如果我只想使用 Top-K 或者 Top-P 应该怎么办？

参考链接

在生成文本时，模型为每个可能的下一个词汇分配一个概率分布，选择下一个词汇的策略直接决定了输出的质量和多样性。以下是几种常见的选择方法:

Greedy Search（贪心搜索）: 每次选择概率最高的词汇。

Beam Search（束搜索）: 保留多个候选序列，平衡生成质量和多样性。

Top-K 采样: 限制候选词汇数量。

Top-P 采样（Nucleus Sampling）: 根据累积概率选择候选词汇，动态调整词汇集。

为了直观叙述，假设我们当前的概率分布为:

词汇 概率 $A$ $0.4$ $B$ $0.3$ $C$ $0.2$ $D$ $0.05$ $\text{<eos>}$ $0.05$

Top-K 采样详解

工作原理

Top-K 采样是一种通过限制候选词汇数量来增加生成文本多样性的方法。在每一步生成过程中，模型只考虑概率最高（Top）的 K 个词汇，然后从这 K 个词汇中根据概率进行采样。K=1 就是贪心搜索。

步骤:

获取概率分布: 模型为每个可能的下一个词汇生成一个概率分布。

筛选 Top-K: 选择概率最高的 K 个词汇，忽略其余词汇。

重新归一化: 将筛选后的 K 个词汇的概率重新归一化，使其总和为 1。

采样: 根据重新归一化后的概率分布，从 Top-K 词汇中随机采样一个词汇作为下一个生成的词。

数学表述

设 $V$ 为词汇表, $P(y|Y)$ 为在给定上下文 $Y$ 下生成词汇 $y$ 的概率。

筛选出概率最高的 K 个词汇，记为

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
)
texts = splitter.split_documents(documents)

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "models/Classical/Yinka"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

/home/ubuntu/anaconda3/envs/torch/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(texts, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})

In [5]:
query = "Top-K和Top-P有什么区别"
retriever_docs = retriever.invoke(query)
for i, doc in enumerate(retriever_docs):
    print(f"Document {i + 1}:")
    print(f"Content: {doc.page_content}\n")

Document 1:
Content: 观察图片可以直观看到:

当 temperature < 1 时，概率分布变得更加尖锐，高概率词更可能被选择，适用于需要高确定性的任务，如生成技术文档或代码。

当 temperature > 1 时，概率分布变得更加平坦，使得低概率词也有更多机会被选中，适用于需要创造性和多样性的任务，如写作或对话生成。

在大模型中的应用

Top-K 和 Top-P 采样是否可以一起使用？

Document 2:
Content: 什么是 Top-K 和 Top-P 采样？Temperature 如何影响生成结果？

在之前的文章中我们探讨了 Beam Search 和 Greedy Search。

现在来聊聊 model.generate() 中常见的三个参数: top-k, top-p 和 temperature。

代码文件下载

目录

采样方法概述

Top-K 采样详解

工作原理

数学表述

代码示例

Top-P 采样详解

工作原理

数学表述

代码示例

Temperature 的作用

工作原理

Document 3:
Content: Top-P 采样详解

工作原理

Top-P 采样（又称 Nucleus Sampling）是一种动态选择候选词汇的方法。与 Top-K 采样不同，Top-P 采样不是固定选择 K 个词汇，而是选择一组累计概率达到 P 的词汇集合（即从高到低加起来的概率）。这意味着 Top-P 采样可以根据当前的概率分布动态调整候选词汇的数量，从而更好地平衡生成的多样性和质量。

步骤:

获取概率分布: 模型为每个可能的下一个词汇生成一个概率分布。

排序概率: 将词汇按照概率从高到低排序。

Document 4:
Content: 参数解释:

top_k=10: 首先限制候选词汇为概率最高的 10 个。

top_p=0.5: 在这 10 个词汇中，从高到低，选择累积概率达到 0.5 的词汇归一化后进行采样。

temperature=0.8: 控制生成的随机性，较低的温度使模型更倾向于高概率词汇。

如果我只想使用 Top-K 或者 Top-P 应该怎么办？

对于只使用 Top-K:

将 top_p 设置为 1（表示不使用 Top-P 采样）。

Document 5:


In [6]:
retriever_docs = vectorstore.similarity_search_with_score(query, 5)
for doc, score in retriever_docs:
    print(f"Score: {score}")
    print(f"Content: {doc.page_content}\n")

Score: 241.8722686767578
Content: 观察图片可以直观看到:

当 temperature < 1 时，概率分布变得更加尖锐，高概率词更可能被选择，适用于需要高确定性的任务，如生成技术文档或代码。

当 temperature > 1 时，概率分布变得更加平坦，使得低概率词也有更多机会被选中，适用于需要创造性和多样性的任务，如写作或对话生成。

在大模型中的应用

Top-K 和 Top-P 采样是否可以一起使用？

Score: 243.77821350097656
Content: 什么是 Top-K 和 Top-P 采样？Temperature 如何影响生成结果？

在之前的文章中我们探讨了 Beam Search 和 Greedy Search。

现在来聊聊 model.generate() 中常见的三个参数: top-k, top-p 和 temperature。

代码文件下载

目录

采样方法概述

Top-K 采样详解

工作原理

数学表述

代码示例

Top-P 采样详解

工作原理

数学表述

代码示例

Temperature 的作用

工作原理

Score: 245.97669982910156
Content: Top-P 采样详解

工作原理

Top-P 采样（又称 Nucleus Sampling）是一种动态选择候选词汇的方法。与 Top-K 采样不同，Top-P 采样不是固定选择 K 个词汇，而是选择一组累计概率达到 P 的词汇集合（即从高到低加起来的概率）。这意味着 Top-P 采样可以根据当前的概率分布动态调整候选词汇的数量，从而更好地平衡生成的多样性和质量。

步骤:

获取概率分布: 模型为每个可能的下一个词汇生成一个概率分布。

排序概率: 将词汇按照概率从高到低排序。

Score: 251.02206420898438
Content: 参数解释:

top_k=10: 首先限制候选词汇为概率最高的 10 个。

top_p=0.5: 在这 10 个词汇中，从高到低，选择累积概率达到 0.5 的词汇归一化后进行采样。

temperature=0.8: 控制生成的随机性，较低的温度使模型更倾向于高概率词汇。

如果我只想使用 Top-K 或者 Top-P 应该怎么办？

对于只

In [26]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_path = 'models/microsoft/Phi-3-mini-4k-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_buffers=True,
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    max_length=4096,
    truncation=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [31]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generator)

In [32]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

In [33]:
query = "Top-K和Top-P有什么区别"
# 获取答案
answer = qa_chain.invoke(query)
print(answer) #回答一坨，还得推理大半天，机器不支持，建议别用本地

{'query': 'Top-K和Top-P有什么区别', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n观察图片可以直观看到:\n\n当 temperature < 1 时，概率分布变得更加尖锐，高概率词更可能被选择，适用于需要高确定性的任务，如生成技术文档或代码。\n\n当 temperature > 1 时，概率分布变得更加平坦，使得低概率词也有更多机会被选中，适用于需要创造性和多样性的任务，如写作或对话生成。\n\n在大模型中的应用\n\nTop-K 和 Top-P 采样是否可以一起使用？\n\n什么是 Top-K 和 Top-P 采样？Temperature 如何影响生成结果？\n\n在之前的文章中我们探讨了 Beam Search 和 Greedy Search。\n\n现在来聊聊 model.generate() 中常见的三个参数: top-k, top-p 和 temperature。\n\n代码文件下载\n\n目录\n\n采样方法概述\n\nTop-K 采样详解\n\n工作原理\n\n数学表述\n\n代码示例\n\nTop-P 采样详解\n\n工作原理\n\n数学表述\n\n代码示例\n\nTemperature 的作用\n\n工作原理\n\nTop-P 采样详解\n\n工作原理\n\nTop-P 采样（又称 Nucleus Sampling）是一种动态选择候选词汇的方法。与 Top-K 采样不同，Top-P 采样不是固定选择 K 个词汇，而是选择一组累计概率达到 P 的词汇集合（即从高到低加起来的概率）。这意味着 Top-P 采样可以根据当前的概率分布动态调整候选词汇的数量，从而更好地平衡生成的多样性和质量。\n\n步骤:\n\n获取概率分布: 模型为每个可能的下一个词汇生成一个概率分布。\n\n排序概率: 将词汇按照概率从高到低排序。\n\n参数解释:\n\ntop_k=10: 首先限制候选词汇为概率最高的 10 个。\n\ntop_p=0.